In [1]:
from utils import *
import datasets
import os
import keras
import numpy as np
import random
from collections import Counter
from itertools import chain
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/home/jose/scratch/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


This model is just taking week4's notebook and plugging it into the chatbot, with some tweaks to make it work with sentences (small changes in embedding size and max_iter).

I tried different things (Amounts of dropout, learning rates and learning rate schedules, GRUs and LSTM cells and I didn't manage to get results better than what you see here, which admittedly are not very good.

In [2]:
#!./download_cornell.sh

In [3]:
data = datasets.readCornellData("data/cornell", max_len=40)

100%|██████████| 83097/83097 [00:03<00:00, 22632.32it/s]


In [4]:
#! ./download_opensubs.sh

In [5]:
#data = datasets.readOpensubsData("data/opensubs/", max_len=20)

In [6]:
# Tokenization

#start_symbol = '^'
end_symbol = '$'
padding_symbol = '#'
start_symbol="^"
all_sym = [start_symbol,end_symbol, padding_symbol] + \
    list(Counter(chain.from_iterable(chain.from_iterable(data))).keys())

In [7]:
word2id = {symbol:i for i, symbol in enumerate('^$#abcdefghijklmnopqrstuvwxyz 0123456789+-')}
id2word = {i:symbol for symbol, i in word2id.items()}

In [8]:
def sentence_to_ids(sentence, word2id, padded_len):
    """ Converts a sequence of symbols to a padded sequence of their ids.
    
      sentence: a string, input/output sequence of symbols.
      word2id: a dict, a mapping from original symbols to ids.
      padded_len: an integer, a desirable length of the sequence.

      result: a tuple of (a list of ids, an actual length of sentence).
    """
    
    sent_ids = [word2id[i] for i in sentence]
    sent_len = len(sent_ids[:padded_len-1])+1
    sent_ids = sent_ids[:padded_len-1]+[word2id["$"]]+[word2id["#"]]*(padded_len-len(sent_ids)-1)
    
    return (sent_ids, sent_len)

In [9]:
def ids_to_sentence(ids, id2word):
    """ Converts a sequence of ids to a sequence of symbols.
    
          ids: a list, indices for the padded sequence.
          id2word:  a dict, a mapping from ids to original symbols.

          result: a list of symbols.
    """
 
    return [id2word[i] for i in ids] 

In [10]:
def batch_to_ids(sentences, word2id, max_len):
    """Prepares batches of indices. 
    
       Sequences are padded to match the longest sequence in the batch,
       if it's longer than max_len, then max_len is used instead.

        sentences: a list of strings, original sequences.
        word2id: a dict, a mapping from original symbols to ids.
        max_len: an integer, max len of sequences allowed.

        result: a list of lists of ids, a list of actual lengths.
    """
    
    max_len_in_batch = min(max(len(s) for s in sentences) + 1, max_len)
    batch_ids, batch_ids_len = [], []
    for sentence in sentences:
        ids, ids_len = sentence_to_ids(sentence, word2id, max_len_in_batch)
        batch_ids.append(ids)
        batch_ids_len.append(ids_len)
    return batch_ids, batch_ids_len

In [11]:
def generate_batches(samples, batch_size=64):
    X, Y = [], []
    for i, (x, y) in enumerate(samples, 1):
        X.append(x)
        Y.append(y)
        if i % batch_size == 0:
            yield X, Y
            X, Y = [], []
    if X and Y:
        yield X, Y

In [12]:
import tensorflow as tf

## Model

In [14]:
class Seq2SeqModel(object):
    def __declare_placeholders(self):
        """Specifies placeholders for the model."""

        # Placeholders for input and its actual lengths.
        self.input_batch = tf.placeholder(
            shape=(None, None), dtype=tf.int32, name='input_batch')
        self.input_batch_lengths = tf.placeholder(
            shape=(None, ), dtype=tf.int32, name='input_batch_lengths')

        # Placeholders for groundtruth and its actual lengths.
        self.ground_truth = tf.placeholder(
            shape=(None, None), dtype=tf.int32, name='ground_truth')
        self.ground_truth_lengths = tf.placeholder(
            shape=(None, ), dtype=tf.int32, name='ground_truth_length')

        self.dropout_ph = tf.placeholder_with_default(
            tf.cast(1.0, tf.float32), shape=[])
        self.learning_rate_ph = tf.placeholder(shape=[], dtype=tf.float32)

    def __create_embeddings(self, vocab_size, embeddings_size):
        """Specifies embeddings layer and embeds an input batch."""

        random_initializer = tf.random_uniform(
            (vocab_size, embeddings_size), -1.0, 1.0)
        self.embeddings = tf.Variable(
            initial_value=random_initializer, dtype=tf.float32)

        # Perform embeddings lookup for self.input_batch.
        self.input_batch_embedded = tf.nn.embedding_lookup(
            self.embeddings, self.input_batch)

    def __build_encoder(self, hidden_size):
        """Specifies encoder architecture and computes its output."""

        # Create GRUCell with dropout.
        encoder_cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(hidden_size),
                                                     input_keep_prob=self.dropout_ph)

        # Create RNN with the predefined cell.
        _, self.final_encoder_state = tf.nn.dynamic_rnn(
            encoder_cell, self.input_batch_embedded, sequence_length=self.input_batch_lengths, dtype=tf.float32)

    def __build_decoder(self, hidden_size, vocab_size, max_iter, start_symbol_id, end_symbol_id):
        """Specifies decoder architecture and computes the output.

            Uses different helpers:
              - for train: feeding ground truth
              - for inference: feeding generated output

            As a result, self.train_outputs and self.infer_outputs are created. 
            Each of them contains two fields:
              rnn_output (predicted logits)
              sample_id (predictions).

        """

        # Use start symbols as the decoder inputs at the first time step.
        batch_size = tf.shape(self.input_batch)[0]
        start_tokens = tf.fill([batch_size], start_symbol_id)
        ground_truth_as_input = tf.concat(
            [tf.expand_dims(start_tokens, 1), self.ground_truth], 1)

        # Use the embedding layer defined before to lookup embedings for ground_truth_as_input.
        self.ground_truth_embedded = tf.nn.embedding_lookup(
            self.embeddings, ground_truth_as_input)

        # Create TrainingHelper for the train stage.
        train_helper = tf.contrib.seq2seq.TrainingHelper(self.ground_truth_embedded,
                                                         self.ground_truth_lengths)

        # Create GreedyEmbeddingHelper for the inference stage.
        # You should provide the embedding layer, start_tokens and index of the end symbol.
        infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding=self.embeddings, start_tokens=word2id["^"]*tf.ones([batch_size], dtype=tf.int32),
                                                                end_token=word2id["$"])

        def decode(helper, scope, reuse=None):
            """Creates decoder and return the results of the decoding with a given helper."""

            with tf.variable_scope(scope, reuse=reuse):
                # Create GRUCell with dropout. Do not forget to set the reuse flag properly.
                decoder_cell = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.GRUCell(hidden_size, reuse=reuse),
                                                             input_keep_prob=self.dropout_ph)

                # Create a projection wrapper.
                decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(
                    decoder_cell, vocab_size, reuse=reuse)

                # Create BasicDecoder, pass the defined cell, a helper, and initial state.
                # The initial state should be equal to the final state of the encoder!
                decoder = tf.contrib.seq2seq.BasicDecoder(
                    decoder_cell, helper=helper, initial_state=self.final_encoder_state)

                # The first returning argument of dynamic_decode contains two fields:
                #   rnn_output (predicted logits)
                #   sample_id (predictions)
                outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, maximum_iterations=max_iter,
                                                                  output_time_major=False, impute_finished=True)

                return outputs

        self.train_outputs = decode(train_helper, 'decode')
        self.infer_outputs = decode(infer_helper, 'decode', reuse=True)

    def __compute_loss(self):
        """Computes sequence loss (masked cross-entopy loss with logits)."""
        
        weights = tf.cast(tf.sequence_mask(
            self.ground_truth_lengths), dtype=tf.float32)
  
        self.loss = tf.contrib.seq2seq.sequence_loss(self.train_outputs.rnn_output,
                                                     self.ground_truth, weights=weights)

    def __perform_optimization(self):
        """Specifies train_op that optimizes self.loss."""

        self.train_op = tf.contrib.layers.optimize_loss(self.loss, optimizer="Adam", learning_rate=self.learning_rate_ph,
                                                        global_step=tf.train.get_global_step(), clip_gradients=1.0)
    def __init__(self, vocab_size, embeddings_size, hidden_size, 
                   max_iter, start_symbol_id, end_symbol_id, padding_symbol_id):

        self.__declare_placeholders()
        self.__create_embeddings(vocab_size, embeddings_size)
        self.__build_encoder(hidden_size)
        self.__build_decoder(hidden_size, vocab_size, max_iter, start_symbol_id, end_symbol_id)

        # Compute loss and back-propagate.
        self.__compute_loss()
        self.__perform_optimization()

        # Get predictions for evaluation.
        self.train_predictions = self.train_outputs.sample_id
        self.infer_predictions = self.infer_outputs.sample_id
    def train_on_batch(self, session, X, X_seq_len, Y, Y_seq_len, learning_rate, dropout_keep_probability):
        feed_dict = {
                self.input_batch: X,
                self.input_batch_lengths: X_seq_len,
                self.ground_truth: Y,
                self.ground_truth_lengths: Y_seq_len,
                self.learning_rate_ph: learning_rate,
                self.dropout_ph: dropout_keep_probability
            }
        pred, loss, _ = session.run([
                self.train_predictions,
                self.loss,
                self.train_op], feed_dict=feed_dict)
        return pred, loss
    
    def predict_for_batch(self, session, X, X_seq_len):
        feed_dict = {self.input_batch: X, self.input_batch_lengths: X_seq_len}
        pred = session.run([
                self.infer_predictions
            ], feed_dict=feed_dict)[0]
        return pred

    def predict_for_batch_with_loss(self, session, X, X_seq_len, Y, Y_seq_len):
        feed_dict = {
                self.input_batch: X,
                self.input_batch_lengths: X_seq_len,
                self.ground_truth: Y,
                self.ground_truth_lengths: Y_seq_len}
        pred, loss = session.run([
                self.infer_predictions,
                self.loss,
            ], feed_dict=feed_dict)
        return pred, loss

## Rest

In [15]:
tf.reset_default_graph()


batch_size = 256
n_epochs = 4
learning_rate = 0.001
dropout_keep_probability = 0.7
max_len = 30

n_step = int(len(data) / batch_size)

model = Seq2SeqModel(vocab_size=len(word2id), embeddings_size=40, max_iter=max_len, hidden_size=256,
                     start_symbol_id=word2id["^"], end_symbol_id=word2id["$"], padding_symbol_id=word2id["#"])

In [16]:
train_set, test_set = train_test_split(data, test_size=0.2, random_state=21)

In [17]:
session = tf.Session()
session.run(tf.global_variables_initializer())


In [31]:
learning_rate = 0.0001
batch_size = 256

In [32]:
for epoch in range(n_epochs):
    random.shuffle(train_set)
    random.shuffle(test_set)
    print('Train: epoch', epoch + 1)
    for n_iter, (X_batch, Y_batch) in enumerate(generate_batches(train_set, batch_size=batch_size)):
        X, X_len = batch_to_ids(X_batch, word2id, max_len=max_len)
        Y, Y_len = batch_to_ids(Y_batch, word2id, max_len=max_len)
        predictions, loss = model.predict_for_batch_with_loss(
            session=session, X=X, X_seq_len=X_len, Y=Y,
            Y_seq_len=Y_len)
        model.train_on_batch(session, X, X_len, Y, Y_len,
                             learning_rate, dropout_keep_probability)
        if n_iter % 10 == 0:
            print("Epoch: [%d/%d], step: [%d/%d], loss: %f" %
                  (epoch + 1, n_epochs, n_iter + 1, n_step, loss))
    
    
    X_sent, Y_sent = next(generate_batches(test_set, batch_size=batch_size))

    X, X_len = batch_to_ids(X_sent, word2id, max_len=max_len)

    Y, Y_len = batch_to_ids(Y_sent, word2id, max_len=max_len)
    predictions, loss = model.predict_for_batch_with_loss(
        session=session, X=X, X_seq_len=X_len, Y=Y,
        Y_seq_len=Y_len)

    print('Test: epoch', epoch + 1, 'loss:', loss,)

Train: epoch 1
Epoch: [1/4], step: [1/310], loss: 1.386444
Epoch: [1/4], step: [11/310], loss: 1.406645
Epoch: [1/4], step: [21/310], loss: 1.408774
Epoch: [1/4], step: [31/310], loss: 1.388344
Epoch: [1/4], step: [41/310], loss: 1.392088
Epoch: [1/4], step: [51/310], loss: 1.438425
Epoch: [1/4], step: [61/310], loss: 1.445774
Epoch: [1/4], step: [71/310], loss: 1.430758
Epoch: [1/4], step: [81/310], loss: 1.367387
Epoch: [1/4], step: [91/310], loss: 1.417184
Epoch: [1/4], step: [101/310], loss: 1.445110
Epoch: [1/4], step: [111/310], loss: 1.449245
Epoch: [1/4], step: [121/310], loss: 1.409753
Epoch: [1/4], step: [131/310], loss: 1.461484
Epoch: [1/4], step: [141/310], loss: 1.411704
Epoch: [1/4], step: [151/310], loss: 1.360555
Epoch: [1/4], step: [161/310], loss: 1.405701
Epoch: [1/4], step: [171/310], loss: 1.422820
Epoch: [1/4], step: [181/310], loss: 1.446037
Epoch: [1/4], step: [191/310], loss: 1.424580
Epoch: [1/4], step: [201/310], loss: 1.446039
Epoch: [1/4], step: [211/310],

In [33]:
predictions = model.predict_for_batch(session, X, X_len)
model_predictions = np.array(
            ["".join(ids_to_sentence(i, id2word)) for i in predictions])

In [34]:
model_predictions

array(['yes i dont know$^^^^^^^^^^^^^^', 'yes i dont know$^^^^^^^^^^^^^^',
       'what do you mean$^^^^^^^^^^^^^', 'what do you mean$^^^^^^^^^^^^^',
       'what do you mean$^^^^^^^^^^^^^', 'i dont know i dont know$^^^^^^',
       'yes i dont know$^^^^^^^^^^^^^^', 'what do you mean$^^^^^^^^^^^^^',
       'i dont know i dont know$^^^^^^', 'i dont know i dont know$^^^^^^',
       'what do you mean$^^^^^^^^^^^^^', 'he was a second of the back$^^',
       'i dont know i dont know$^^^^^^', 'i dont know i dont know$^^^^^^',
       'i dont know i dont know$^^^^^^', 'he was a second of the back$^^',
       'what do you mean$^^^^^^^^^^^^^', 'what do you mean$^^^^^^^^^^^^^',
       'i dont know i dont know$^^^^^^', 'i dont know i dont know$^^^^^^',
       'i dont know i dont know$^^^^^^', 'i dont know i dont know$^^^^^^',
       'i dont know i dont know$^^^^^^', 'i dont know i dont know$^^^^^^',
       'what do you mean$^^^^^^^^^^^^^', 'i dont know i dont know$^^^^^^',
       'i dont know i don

In [35]:
def reply(question,word2id,max_len,model,id2word):

    ids, ids_len = sentence_to_ids(question,word2id,padded_len=max_len)
    ids=np.array(ids).reshape(1,len(ids))

    ids_len=np.array(ids_len).reshape(1)
    predictions = model.predict_for_batch(session, ids, ids_len)
    return "".join(ids_to_sentence(predictions[0], id2word)).replace("$","").capitalize()

### Good responses

In [37]:
reply("hi",word2id,15,model,id2word)

'Hi'

In [42]:
reply("how are you",word2id,15,model,id2word)

'I dont know i dont know'

In [52]:
reply("blerg",word2id,15,model,id2word)

'What do you mean'

### Not so good responses

In [58]:
reply("tell me about you",word2id,15,model,id2word)

'I dont know i dont know'

In [63]:
reply("why",word2id,15,model,id2word)

'I dont know what i said it was'

In [65]:
reply("hello",word2id,15,model,id2word)

'He was a second of the back'

In [38]:
saver = tf.train.Saver()

In [39]:
saver.save(session,"../model/chatbot_model")

'../model/chatbot_model'

## Chatbot part

This bit goes in the existing chatbot code, in a tfchatbot.py file to be imported

In [ ]:
import tensorflow as tf
from utils import *
end_symbol = '$'
padding_symbol = '#'
start_symbol="^"
word2id = {symbol:i for i, symbol in enumerate('^$#abcdefghijklmnopqrstuvwxyz 0123456789+-')}
id2word = {i:symbol for symbol, i in word2id.items()}
max_len = 30

sess = tf.Session()
new_saver = tf.train.import_meta_graph('model/chatbot_model.meta')
new_saver.restore(sess, tf.train.latest_checkpoint('model/'))
sess.run(tf.local_variables_initializer())
def reply(question, word2id, max_len, id2word, session):
    input_batch = tf.get_default_graph().get_tensor_by_name("input_batch:0")
    input_batch_len = tf.get_default_graph().get_tensor_by_name("input_batch_lengths:0")
    infer_predictions = tf.get_default_graph().get_tensor_by_name(
        "decode_1/decoder/transpose_1:0")

    question = text_prepare(question)
    ids, ids_len = sentence_to_ids(question, word2id, padded_len=max_len)
    ids = np.array(ids).reshape(1, len(ids))

    ids_len = np.array(ids_len).reshape(1)
    predictions = session.run([
        infer_predictions
    ], feed_dict={input_batch: ids, input_batch_len: ids_len})[0]
    return "".join(ids_to_sentence(predictions[0], id2word)).replace("$", "").capitalize()
reply("hi",word2id,max_len,id2word,sess)